# Generate Map

> Class that generates the data map

In [ ]:
#| default_exp app

In [ ]:
#| export
from nbdev.showdoc import *
try:
    from SolomonIslandsDataMap.app_data import stored_data, sidebar
except: 
    from app_data import stored_data, sidebar

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # Unless this is used graphs will not be dynamic?
import numpy as np
from fastcore.test import *
from dash import page_container, Dash, dcc, Output, Input, State, html, Patch, page_registry, ctx  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
from dash_bootstrap_templates import load_figure_template
from git import Repo
import json


## Setup App and Server

Setup the app and global variables

In [ ]:
#| export

# Find the absoulte path to the pages folder
repo = Repo('.', search_parent_directories=True)
fp = str(repo.working_tree_dir) + "/SolomonIslandsDataMap/pages/"

try:
    app = Dash(__name__, external_stylesheets=[dbc.themes.MINTY], use_pages=True, pages_folder = fp)
except:
    # When running in a notebook, the below trick should get the notebook to still execute
    import __main__ as main
    main.__file__ = "main_file"
    app = Dash(__name__, external_stylesheets=[dbc.themes.MINTY], use_pages=True, pages_folder = fp)
server = app.server
load_figure_template("minty")

In [ ]:
print(fp)

/home/tom/git/SolomonIslandsDataMap/SolomonIslandsDataMap/pages/


Create a navbar

In [ ]:
#| export
pages = {}
for page in page_registry.values():
    pages[page["title"]] = page["relative_path"]
    
navbar = dbc.NavbarSimple(
    children=[
        # TODO make this a dbc tab with calbback to make it active
        dbc.NavItem(dbc.NavLink('Census Map', href=pages['Census Map'], active=True)),
        dbc.NavItem(dbc.NavLink('Election Map', href=pages['Election Map'])),
        dbc.NavItem(dbc.NavLink('Population Projections', href=pages['Population Projections'])),
        dbc.NavItem(dbc.NavLink('Data Table', href=pages['Data Table'])),
        dbc.DropdownMenu(
            children=[
                #dbc.DropdownMenuItem("More pages coming soon", header=True),
                
                dbc.DropdownMenuItem('Census Map', href=pages['Census Map']),
                dbc.DropdownMenuItem('Election Map', href=pages['Election Map']),
                dbc.DropdownMenuItem('Population Projections', href=pages['Population Projections']),
                dbc.DropdownMenuItem('Data Table', href=pages['Data Table']),
            ],
            nav=True,
            in_navbar=True,
            label="Select Page",
        ),
    ],
    brand="Solomon Islands Data Explorer",
    brand_href="#",
    color="primary",
    #dark=True,
    class_name="navbar navbar-expand-lg bg-primary"
)


Import the sidebar

## Create the Layout

In [ ]:
#| export
app.layout = dbc.Container([
                dbc.Row([
                    navbar
                ]),
                dbc.Row(
                [dbc.Col(sidebar, width = 2),
                dbc.Col([
                    page_container, ], width = 10),
                ], justify = 'center'),
                #stored_data, # removing stored data for now, note necessary unless using for clientside callbacks                                    
                ], fluid = True)

## Run the dash app

In [ ]:
#| export
#| server
# Run app
if __name__=='__main__':
    try:
        app.run_server(debug=True, port=9999) # Random int mitigates port collision
    except:
        print("Cannot run server here")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

Func: update_geography
Func: update_map
Updating map for census
Func: map_click
Func: bar_click
Func: map_selections
Func: update_bargraph
Func: update_geography
Func: update_election_bargraph
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File ~/git/SolomonIslandsDataMap/SolomonIslandsDataMap/dash_components.py:183, in election_bar_plot(
    sol_geo=<SolomonIslandsDataMap.load_data.SolomonGeo object>,
    geo_filter='Constituency',
    year=2024,
    election='National Parliament',
    location='Central Honiara',
    type_filter='Total'
)
    177 fig = go.Figure()
    178 measures = list(df.columns)
    179 fig.add_trace(go.Bar(
    180     x = df.loc[df['loc_name'] == location, 'Candidate'].values,
    181     y = df.loc[df['loc_name'] == location, type_filter].values,
    182     name = location,
--> 183     marker = dict(color = list(map(lambda y: sol_geo.colormap[y], df.loc[df[